In [1]:
!conda install psycopg2 -y

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - psycopg2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libpq-15.1                 |       hd7ccf0a_3         2.4 MB  conda-forge
    psycopg2-2.9.3             |  py310hc417edf_2         171 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

  libpq              conda-forge/linux-aarch64::libpq-15.1-hd7ccf0a_3 
  psycopg2           conda-forge/linux-aarch64::psycopg2-2.9.3-py310hc417edf_2 



psycopg2-2.9.3       | 171 KB    |                                       |   0% 
psycopg2-2.9.3       | 171 KB    | ###4                                  |   9% 
psycopg2-2.9.3       | 171 KB    | ##################################### | 100% 


In [3]:
import psycopg2
import csv
from glob import glob
from datetime import datetime


def connect_postgres(host: str = 'postgres', user: str = 'postgres', pwd: str = 'postgres', port: int = 5432, db='trip'):
    try:
        conn = psycopg2.connect(f'postgresql://{user}:{pwd}@{host}:{port}/{db}')
        return conn
    except psycopg2.Error as e:
        print(f'Had problem connecting with error {e}.')


def pull_data_files(loc: str = './trip-data/Divvy_Trips_2015_Q3Q4/*Trips*.csv') -> list:
    files = glob(loc)
    return files


def read_file(file: str) -> list:
    with open(file) as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)
        rows = [row for row in reader]
    return rows


def insert_rows(rows: list, conn: object) -> None:
    cur = conn.cursor()
    i = 0
    for row in rows:
        i = i + 1;
        cur.execute("INSERT INTO trip_data VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                    (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9]
                        , row[10], row[11]))
        if not i % 10:
            conn.commit()
    conn.commit()


if __name__ == '__main__':
    t1 = datetime.now()
    conn = connect_postgres()
    files = pull_data_files()
    for file in files:
        print(f"working on file {file}")
        rows = read_file(file)
        insert_rows(rows, conn)
        print(f"finished with file {file}")
    # conn.close()
    t2 = datetime.now()
    x = t2 - t1
    print(f"time was {x}")

working on file ./trip-data/Divvy_Trips_2015_Q3Q4/Divvy_Trips_2015_07.csv
finished with file ./trip-data/Divvy_Trips_2015_Q3Q4/Divvy_Trips_2015_07.csv
working on file ./trip-data/Divvy_Trips_2015_Q3Q4/Divvy_Trips_2015_08.csv


KeyboardInterrupt: 